In [1]:
import numpy as np
from numba import njit
from scipy.linalg import cholesky
import anDiffReg as an

In [2]:
# exemplary data: simulated fractional Brownian motion

n = 10**4 # number of trajectories
H, D = 0.4, 1 # FBM parameters: Hurst index H and diffusivity D
ln = 100 # trajectory length
dt = 1 # time interval
ts = dt * np.arange(1, ln+1)


def K(s, t): # FBM covariance
    return D * (t**(2*H) + s**(2*H) - np.abs(s - t)**(2*H))

S = np.array([[K(s, t) for t in ts] for s in ts])
A = cholesky(S, lower=False)
xi = np.random.randn(len(ts), n)
X = A.T @ xi
xi = np.random.randn(len(ts), n) 
Y = A.T @ xi

In [3]:
## TA-MSD analysis

msd = an.tamsd(np.dstack((X,Y))) # TA-MSD of 2D traj

In [18]:
ols = an.fit_ols(msd, 2, dt)

In [19]:
np.mean(ols, axis = 1)

array([-6.12241976e-04,  7.90298243e-01])

In [20]:
np.cov(ols)

array([[ 0.00247726, -0.00199446],
       [-0.00199446,  0.01144049]])

In [4]:
gls, covGLS = an.fit_gls(msd, 2, dt, np.full(10**4,0.8)) 

d:\GitHub\DH\anDiffReg.py:101: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'C', False, aligned=True), Array(float64, 2, 'A', False, aligned=True))
  gR = np.linalg.inv(Ts.T @ iC[:, :, j0] @ Ts) @ Ts.T @ iC[:, :, j0]


In [5]:
np.mean(gls, axis = 1)

array([-4.05795993e-04,  7.99860115e-01])

In [7]:
np.cov(gls)

array([[ 0.00196419, -0.00072089],
       [-0.00072089,  0.00762311]])

In [9]:
glsP, covGLSP = an.fit_gls(msd[:,:100], 2, dt, np.full(10**4,0.8), precompute = False)
# covariance computed for each trajectory, could be slow if there are many

In [ ]:
## trajectories with noise

D, sigma = 5, 1

X2 = np.sqrt(D)*X + sigma*np.random.randn(len(ts), n)
msd2 = an.tamsd(X2)

In [16]:
gls2, covGLS2 = an.fit_gls(msd2, 1, 1, np.full(10**4,2*H), np.full(10**4,D), sigma)
# initial estimates of α, D and σ must be used 

In [17]:
np.mean(gls2, axis=1)

array([0.69788459, 0.79886057])

In [18]:
np.cov(gls2)

array([[ 0.00604832, -0.00358441],
       [-0.00358441,  0.01846064]])